## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...
1,2025-07-11-19-39-41 +0000,nyt,Evidence Supports War Crimes Allegations in Da...,https://www.nytimes.com/2025/07/11/world/afric...
2,2025-07-11-19-22-18 +0000,bbc,Trump threatens 35% tariffs on Canadian goods,https://www.bbc.com/news/articles/cvg819n954mo
3,2025-07-11-19-04-37 +0000,nyt,Trump Says NATO Countries Will Buy Weapons to ...,https://www.nytimes.com/2025/07/11/world/europ...
4,2025-07-11-19-00-15 +0000,nyt,Police Investigate Burning of Migrant-Boat Eff...,https://www.nytimes.com/2025/07/11/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,18
3,tariff,8
2,canada,7
17,tariffs,5
47,cuts,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
25,2025-07-11-13-41-59 +0000,cbc,CUSMA-compliant goods exempt from Trump's late...,https://www.cbc.ca/news/business/trump-tariff-...,47
0,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...,43
21,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,43
28,2025-07-11-13-13-32 +0000,cbc,RECAP | Trump's latest tariff threat sends tra...,https://www.cbc.ca/news/politics/livestory/rec...,39
9,2025-07-11-17-52-39 +0000,nyt,"As Trump Sows Tariff Confusion, Rules of Globa...",https://www.nytimes.com/2025/07/11/world/europ...,37


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
25,47,2025-07-11-13-41-59 +0000,cbc,CUSMA-compliant goods exempt from Trump's late...,https://www.cbc.ca/news/business/trump-tariff-...
21,25,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
11,17,2025-07-11-17-37-00 +0000,wsj,Nuclear power exemplifies how revamping dated ...,https://www.wsj.com/economy/trumps-unsung-econ...
30,14,2025-07-11-13-02-28 +0000,bbc,National Trust blames tax rises as it cuts 550...,https://www.bbc.com/news/articles/clym8029nnvo
46,14,2025-07-11-05-25-20 +0000,nyt,Rare Earths Mining in Myanmar Is Polluting Tha...,https://www.nytimes.com/2025/07/11/world/asia/...
29,13,2025-07-11-13-07-16 +0000,nyt,"As Climate Change Heats Up Europe’s Summers, A...",https://www.nytimes.com/2025/07/11/theater/avi...
41,12,2025-07-11-09-00-37 +0000,nyt,"In ‘Apocalypse in the Tropics,’ Director Petra...",https://www.nytimes.com/2025/07/11/movies/petr...
47,12,2025-07-11-05-00-11 +0000,nyt,"Friday Briefing: Iran Has Some Uranium Left, I...",https://www.nytimes.com/2025/07/11/briefing/ir...
4,10,2025-07-11-19-00-15 +0000,nyt,Police Investigate Burning of Migrant-Boat Eff...,https://www.nytimes.com/2025/07/11/world/europ...
27,10,2025-07-11-13-26-30 +0000,nyt,Manitoba Declares Second Wildfire Emergency in...,https://www.nytimes.com/2025/07/11/world/canad...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
